# Initialization

## Importing dependencies

Install the API for eurostat

In [1]:
# Install the Eurostat API wrapper ( the official Python package)
# Allows for easy fetching of datasets like the 'urb_percep' we will be using directly from Eurostat
# Documentation: https://pypi.org/project/eurostat/, we use % as that is what Jupyter uses
%pip install eurostat


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Call imports

In [2]:
import eurostat # loading the Eurostat API wrapper we just installed above, gives access to the commands we will be using
import pandas as pd # self-explanatory
import plotly.graph_objects as go
import geopandas as gpd # map display
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from functools import reduce
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Declaring functions

### Functions used for ETL

In [3]:
def add_eurovoc_region(df):
    """
    Add to the first column a country mapping using the lookup table found in https://en.wikipedia.org/wiki/EuroVoc

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame that has a Country column as the first column.


    Returns
    -------
    pd.DataFrame
        The same DataFrame with the new regions column.
    """
    # EuroVoc classification mapping
    eurovoc_map = {
        # Central and Eastern Europe
        'Albania': 'Central and Eastern Europe', 'Armenia': 'Central and Eastern Europe',
        'Azerbaijan': 'Central and Eastern Europe', 'Belarus': 'Central and Eastern Europe',
        'Bosnia and Herzegovina': 'Central and Eastern Europe', 'Bulgaria': 'Central and Eastern Europe',
        'Croatia': 'Central and Eastern Europe', 'Czech Republic': 'Central and Eastern Europe',
        'Georgia': 'Central and Eastern Europe', 'Hungary': 'Central and Eastern Europe',
        'Kosovo': 'Central and Eastern Europe', 'Moldova': 'Central and Eastern Europe',
        'Montenegro': 'Central and Eastern Europe', 'North Macedonia': 'Central and Eastern Europe',
        'Poland': 'Central and Eastern Europe', 'Romania': 'Central and Eastern Europe',
        'Russia': 'Central and Eastern Europe', 'Serbia': 'Central and Eastern Europe',
        'Slovakia': 'Central and Eastern Europe', 'Slovenia': 'Central and Eastern Europe',
        'Ukraine': 'Central and Eastern Europe',

        # Northern Europe
        'Denmark': 'Northern Europe', 'Estonia': 'Northern Europe', 'Finland': 'Northern Europe',
        'Iceland': 'Northern Europe', 'Latvia': 'Northern Europe', 'Lithuania': 'Northern Europe',
        'Norway': 'Northern Europe', 'Sweden': 'Northern Europe',

        # Southern Europe
        'Cyprus': 'Southern Europe', 'Greece': 'Southern Europe', 'Holy See': 'Southern Europe',
        'Italy': 'Southern Europe', 'Malta': 'Southern Europe', 'Portugal': 'Southern Europe',
        'San Marino': 'Southern Europe', 'Spain': 'Southern Europe', 'Turkey': 'Southern Europe',

        # Western Europe
        'Andorra': 'Western Europe', 'Austria': 'Western Europe', 'Belgium': 'Western Europe',
        'France': 'Western Europe', 'Germany': 'Western Europe', 'Ireland': 'Western Europe',
        'Liechtenstein': 'Western Europe', 'Luxembourg': 'Western Europe', 'Monaco': 'Western Europe',
        'Netherlands': 'Western Europe', 'Switzerland': 'Western Europe', 'United Kingdom': 'Western Europe'
    }

    # Add new column based on mapping
    df.insert(
        loc=1,
        column='EuroVoc Region',
        value=df.iloc[:, 0].map(eurovoc_map).fillna('Unknown')
    )
    return df

In [4]:
def align_on_city_period(df1, df2):
    """
    Align two dataframes by retaining only rows with shared cities\TIME_PERIOD values.

    Parameters
    ----------
    df1 : pd.DataFrame
        First dataframe with a 'cities\TIME_PERIOD' column.
    df2 : pd.DataFrame
        Second dataframe with a 'cities\TIME_PERIOD' column.

    Returns
    -------
    tuple of pd.DataFrame
        Filtered versions of df1 and df2 with matching cities\TIME_PERIOD entries.
    """
    # Find common keys
    common_keys = set(df1["cities\TIME_PERIOD"]).intersection(df2["cities\TIME_PERIOD"])

    # Filter both dataframes
    df1_filtered = df1[df1["cities\TIME_PERIOD"].isin(common_keys)].copy()
    df2_filtered = df2[df2["cities\TIME_PERIOD"].isin(common_keys)].copy()

    return df1_filtered, df2_filtered


In [5]:
def aggregate_by_country(df):
    """
    Aggregate numeric columns by 'Country', removing city-level rows.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing at least 'City/LAU' and 'Country'.

    Returns
    -------
    pd.DataFrame
        DataFrame with one row per Country and numeric columns averaged.
        Non-numeric columns (except 'Country') are dropped.
    """
    # Defensive copy to protect the original dataframe
    df = df.copy()

    # Drop the city identifier column
    df = df.drop(columns=["City/LAU"])

    # Select numeric columns only (keeps Country for grouping)
    numeric_cols = df.select_dtypes(include="number").columns.tolist()

    # Group by Country and compute mean of numeric columns
    aggregated = df.groupby("Country", as_index=False)[numeric_cols].mean()

    return aggregated


In [6]:
def combine_yearly_data(dfs:list[pd.DataFrame], labels:list[str], year:str):
    """
    Combine values from four datasets for a given year,
    assuming cities\TIME_PERIOD is consistent across all.

    Parameters
    ----------
    dfs : list of pd.DataFrame
        List of four dataframes, each with 'cities\TIME_PERIOD' and year columns.
    labels : list of str
        List of labels to name each value column in the output.        
    year : str
        Year column to extract (e.g., "20XX").

    Returns
    -------
    pd.DataFrame
        A hard copy of the combined dataframe around cities\TIME_PERIOD.
    """

     # Rename each year column to the corresponding label
    renamed = [
        df[["cities\TIME_PERIOD", year]].rename(columns={year: labels[i]}).copy(deep=False)
        for i, df in enumerate(dfs)
    ]

    # Merge all on cities\\TIME_PERIOD
    merged = renamed[0]
    for df in renamed[1:]:
        merged = merged.merge(df, on="cities\TIME_PERIOD", how="inner")

    return merged.copy()


In [7]:
def convert_regions(df, column_name: str, conversion_dict: dict[str,int]):
    """
    Converts string entries in a specified column of a DataFrame to integers
    using a provided dictionary.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - column_name (str): The name of the column to convert.
    - conversion_dict (dict): A dictionary mapping string values to integers.

    Returns:
    - pd.DataFrame: The modified DataFrame with converted values.
    """
    df[column_name] = df[column_name].map(conversion_dict).fillna(df[column_name])
    return df

In [8]:
def load_city_key(path):
    """
    Read  the keys tsv file and return a DataFrame keyed by CODE
    containing only the English label.

    Returns
    -------
    pd.DataFrame
        Index: CODE (string)
        Column: Label - English (string)
    """
    # Read TSV, explicitly select columns to avoid loading unnecessary data
    df = pd.read_csv(path, sep="\t", usecols=["CODE", "Label - English"], dtype=str, encoding="utf-8", header=0)

    # Strip whitespace from strings and drop rows with missing CODE or label
    df["CODE"] = df["CODE"].str.strip()
    df["Label - English"] = df["Label - English"].str.strip()
    df = df.dropna(subset=["CODE", "Label - English"])

    # Remove possible duplicate CODEs keeping the first occurrence
    df = df.drop_duplicates(subset=["CODE"], keep="first")

    # Set CODE as the index and return
    df = df.set_index("CODE")
    return df

In [9]:
def filter_by_city_list(df, city_list: list[str]):
    """
    Filter dataframe to include only rows where 'City/FAU' is in the given city list.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with a 'City/FAU' column.
    city_list : list of str
        List of city names to retain.

    Returns
    -------
    pd.DataFrame
        Filtered dataframe with only matching cities.
    """
    return df[df["City/LAU"].isin(city_list)].copy(deep = False)


In [10]:
def filter_by_indicator(code: str, keep_cols: list[str], df, code_indicator = "indic_ur"):
    """
    Filter full_data by indic_ur code and return only cities\TIME_PERIOD and year columns.

    Parameters
    ----------
    code : str
        The indicator code to filter by (e.g., "PS351#V").
    code_indicator : str, optional
        The column name for the indicator code, by default "indic_ur".
    keep_cols : list[str]
        The columns to keep in the filtered dataframe.
    df : pd.DataFrame
        The full_data dataframe with expected columns.

    Returns
    -------
    pd.DataFrame
        Filtered dataframe with cities\TIME_PERIOD and year columns only.
    """
    
    # Filter rows and return selected columns
    return df[df[code_indicator] == code][keep_cols].copy(deep=False)


In [11]:
def merge_datasets_CityCountry(dataset_dict: dict[str, pd.DataFrame]):
    """
    Merges multiple DataFrames on the 'City/LAU' and 'Country' columns.
    
    Parameters:
        dataset_dict (dict): A dictionary where keys are dataset names and values are DataFrames
                             with 'City/LAU', 'Country', and one other column.
                             
    Returns:
        pd.DataFrame: A merged DataFrame with 'Country', 'City/LAU', and one column per dataset.
    """
    renamed_dfs = []
    
    for name, df in dataset_dict.items():
        # Identify the value column (not 'City/LAU' or 'Country')
        value_col = [col for col in df.columns if col not in ["City/LAU", "Country"]][0]
        # Rename the value column to the dataset name
        renamed_df = df.rename(columns={value_col: name})
        renamed_dfs.append(renamed_df)
    
    # Merge all DataFrames on 'City/LAU' and 'Country'
    merged_df = reduce(lambda left, right: pd.merge(left, right, on=["City/LAU", "Country"], how="outer"), renamed_dfs)
    
    # Reorder columns: 'Country', 'City/LAU', then dataset columns
    cols = merged_df.columns.tolist()
    reordered_cols = ['Country', 'City/LAU'] + [col for col in cols if col not in ['Country', 'City/LAU']]
    merged_df = merged_df[reordered_cols]
    
    return merged_df

In [12]:
def rate_of_change_agreement(df1, df2):
    """
    Calculate the rate of change in agreement between two dataframes without altering the inputs.

    Parameters
    ----------
    df1 : pd.DataFrame
        First dataframe with multiple 'agree' columns.
    df2 : pd.DataFrame
        Second dataframe with multiple 'agree' columns.

    Returns
    -------
    pd.DataFrame
        A merged dataframe with the change in precieved agreement for each city.
    """
    # Defensive copy to protect the original dataframes
    df1 = df1.copy()
    df2 = df2.copy()

    # Find the totale agreement for each city in both dataframes
    df1["Total Agreement"] = df1["Strongly agree"] + df1["Somewhat agree"]
    df2["Total Agreement"] = df2["Strongly agree"] + df2["Somewhat agree"]

    # Merge the dataframes on cities\TIME_PERIOD
    merged = df1[["Country", "City/LAU", "Total Agreement"]].merge(
        df2[["Country", "City/LAU", "Total Agreement"]],
        on=["Country", "City/LAU"],
        suffixes=("_2019", "_2023")
    )

    # Find the change in percentage
    merged["Change in Agreement"] = merged["Total Agreement_2023"] - merged["Total Agreement_2019"]

    return merged

### Functions used for the maps 

In [13]:
def map_change(df, value_col="Change in Agreement", title="Change in Perceived Urban Safety (2019–2023)"):
    """
    Map of change by country.
    Works whether df['Country'] holds ISO-2 codes (e.g., 'AT') or full names (e.g., 'Austria').
    """

    # Load Natural Earth countries (GeoPandas ≥1.0 no bundled datasets)
    world = gpd.read_file(
        "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
    )

    # Column name helpers (handles casing)
    cols = {c.lower(): c for c in world.columns}
    cont_col = cols.get("continent", "CONTINENT")
    iso_col  = cols.get("iso_a2", "ISO_A2")
    # candidate name columns in Natural Earth (varies by version)
    name_cols = [c for c in ["ADMIN", "NAME_EN", "NAME"] if c in world.columns]

    europe = world[world[cont_col] == "Europe"].copy()

    # Decide how to join: codes vs names
    country_vals = df["Country"].astype(str)
    looks_like_iso2 = country_vals.str.match(r"^[A-Z]{2}$").all()

    if looks_like_iso2:
        left_on, right_on = iso_col, "Country"
    else:
        # try to join on the first available name column
        if not name_cols:
            raise KeyError("No country name column found in Natural Earth data.")
        left_on, right_on = name_cols[0], "Country"

    merged = europe.merge(df[[right_on, value_col]], how="left", left_on=left_on, right_on=right_on)

    # Optional: show which countries didn’t match
    missing = (merged[merged[value_col].isna()][left_on]
               .dropna().astype(str).unique().tolist())
    if missing:
        print("No match for:", ", ".join(sorted(missing)))

    plt.figure(figsize=(12, 6))
    ax = plt.gca()
    merged.plot(column=value_col, cmap="RdYlGn",
                linewidth=0.6, edgecolor="black",
                legend=True, legend_kwds={"label": "Change in Agreement (%)"},
                ax=ax,
                vmin=-10, vmax=10   # makes gradient steeper + focused
                )

    # Zoom in on Europe (bounding box)
    ax.set_xlim(-10, 35)   # longitude range
    ax.set_ylim(35, 72)    # latitude range


    ax.set_title(title, fontsize=14, pad=10)
    ax.axis("off")
    plt.show()


In [14]:
def map_current(df, value_col="Total Agreement_2023", 
                title="Perceived Urban Safety — 2023"):
    """
    Map current safety agreement levels by country.
    Uses the same structure as map_change but with a sequential colormap.
    """

    world = gpd.read_file(
        "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
    )

    cols = {c.lower(): c for c in world.columns}
    cont_col = cols.get("continent", "CONTINENT")
    iso_col  = cols.get("iso_a2", "ISO_A2")
    name_cols = [c for c in ["ADMIN", "NAME_EN", "NAME"] if c in world.columns]

    europe = world[world[cont_col] == "Europe"].copy()

    looks_like_iso2 = df["Country"].astype(str).str.match(r"^[A-Z]{2}$").all()
    if looks_like_iso2:
        left_on, right_on = iso_col, "Country"
    else:
        left_on, right_on = name_cols[0], "Country"

    merged = europe.merge(df[[right_on, value_col]], how="left",
                          left_on=left_on, right_on=right_on)

    plt.figure(figsize=(9, 7))
    ax = plt.gca()

    #  Sequential green palette for current safety levels
    merged.plot(
        column=value_col,
        cmap="RdYlGn",
        linewidth=0.6,
        edgecolor="black",
        legend=True,
        legend_kwds={"label": f"{value_col} (%)"},
        ax=ax,
        vmin=60, vmax=95  #  fixed range for consistent interpretation
    )

    ax.set_xlim(-10, 35)
    ax.set_ylim(35, 72)
    ax.set_title(title, fontsize=14, pad=10)
    ax.axis("off")
    plt.show()


### Functions used for tourism cross reference data

In [15]:
def load_tourism_annual_wide(path, years=(2019, 2023)):
    """
    Parse Eurostat 'tour_occ' wide TSV where col0 is '...geo\\TIME_PERIOD'
    and month columns look like '2019-01 ' etc. Returns one row per ISO-2 'geo':
    ['geo', 'Nights_<year>', ...]
    """
    df = pd.read_csv(path, sep="\t", dtype=str, na_values=":")

    # Strip BOM + whitespace from column names (your file has trailing spaces)
    clean_cols = [c.encode('utf-8').decode('utf-8').strip() for c in df.columns]
    df.columns = clean_cols

    # First column packs dims + '\TIME_PERIOD'
    key_col = df.columns[0]
    left = df[key_col].astype(str).str.split("\\", n=1).str[0]
    df["geo"] = left.str.split(",").str[-1].str.strip()

    # Month columns like 2019-01, 2023-12 (after stripping)
    month_cols = [c for c in df.columns if re.fullmatch(r"\d{4}-\d{2}", c) or re.fullmatch(r"\d{4}M\d{2}", c)]
    if not month_cols:
        raise ValueError("No monthly columns found after cleaning headers.")

    # Convert month cells to numeric (strip flags like ' e', ' c')
    def _to_num(s):
        return pd.to_numeric(pd.Series(s).astype(str).str.extract(r"([\-0-9\.]+)")[0], errors="coerce")
    df[month_cols] = df[month_cols].apply(_to_num, axis=0)

    # Sum months for each requested year
    out = df[["geo"]].copy()
    for y in years:
        ycols = [c for c in month_cols if c.startswith(str(y))]
        out[f"Nights_{y}"] = df[ycols].sum(axis=1)

    # Collapse any duplicates from other dims -> one row per geo
    out = out.groupby("geo", as_index=False)[[f"Nights_{y}" for y in years]].sum()
    return out

In [16]:
def prepare_tourism_change(annual_df):
    """
    Map ISO-2 -> country names and compute % change (2023 vs 2019).
    Returns: ['Country','Nights_2019','Nights_2023','Tourism Δ (%)']
    """
    world = gpd.read_file("https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip")
    iso_col  = "ISO_A2" if "ISO_A2" in world.columns else "iso_a2"
    name_col = "ADMIN" if "ADMIN" in world.columns else "NAME"
    iso_to_name = dict(zip(world[iso_col], world[name_col]))

    out = annual_df.copy()
    out["Country"] = out["geo"].map(iso_to_name)
    out = out.dropna(subset=["Country", "Nights_2019", "Nights_2023"])

    out["Tourism Δ (%)"] = (out["Nights_2023"] - out["Nights_2019"]) / out["Nights_2019"] * 100
    return out[["Country", "Nights_2019", "Nights_2023", "Tourism Δ (%)"]]

### Functions used for plots

In [17]:
def plot_city_extremes(df, n=10):
    # pick top/bottom movers
    top = df.nlargest(n, "Change in Agreement")
    bottom = df.nsmallest(n, "Change in Agreement")
    sample = pd.concat([top, bottom]).copy()
    # nicer label
    sample["City label"] = sample["City/LAU"] + " — " + sample["Country"]

    plt.figure(figsize=(10, 7))
    sns.barplot(
        data=sample.sort_values("Change in Agreement"),
        x="Change in Agreement", y="City label", hue="Country", dodge=False
    )
    plt.axvline(0, color="gray", ls="--", lw=1)
    plt.title("Cities with the Biggest Shifts in Perceived Safety (2019→2023)")
    plt.xlabel("Change in Agreement (%)")
    plt.ylabel("")
    plt.legend(title="Country", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

In [18]:
def plot_city_vs_country(
    df,
    city_col="City/LAU",
    country_col="Country",
    value_col="Change in Agreement",
    annotate=10,        # how many largest outliers to label (abs deviation)
    figsize=(8, 6),
):
    """
    Scatter: city change vs country average change.
    Points above the diagonal improved more than their country's average; below = worse.
    Returns the dataframe with country averages and deviation, sorted by |deviation|.
    """
    # country average change
    country_avg = (df.groupby(country_col, as_index=False)[value_col]
                     .mean()
                     .rename(columns={value_col: "Country Δ"}))

    # join back to cities + deviation from country trend
    data = df.merge(country_avg, on=country_col, how="left").copy()
    data["Deviation"] = data[value_col] - data["Country Δ"]

    # figure
    plt.figure(figsize=figsize)
    ax = sns.scatterplot(
        data=data, x="Country Δ", y=value_col, hue=country_col, legend=False, s=50
    )

    # 45° line (city == country)
    m = min(data["Country Δ"].min(), data[value_col].min())
    M = max(data["Country Δ"].max(), data[value_col].max())
    ax.plot([m, M], [m, M], color="gray", ls="--", lw=1)

    # label top outliers by |Deviation|
    if annotate and annotate > 0:
        lab = data.reindex(data["Deviation"].abs().nlargest(annotate).index)
        for _, r in lab.iterrows():
            ax.text(r["Country Δ"], r[value_col], f" {r[city_col]}", va="center", fontsize=9)

    ax.set_title("City Change vs Country Average (2019→2023)")
    ax.set_xlabel("Country average change (%)")
    ax.set_ylabel("City change (%)")
    plt.tight_layout()
    plt.show()

    return data.sort_values("Deviation", key=lambda s: s.abs(), ascending=False)

In [19]:
def plot_dumbbell_agreement(df,
                            category_col,
                            left_col="Total Agreement_2019",
                            right_col="Total Agreement_2023",
                            left_color="#FF8C42",   # orange tone
                            right_color="#1F77B4",  # blue (plotly default-ish)
                            increase_color="#2CA02C",# green for increases
                            decrease_color="#D62728",# red for decreases
                            marker_size=10,
                            figsize_height_per_row=30):
    """
    Draw a horizontal dumbbell plot with Plotly showing left_col -> right_col per category.
    - df: pandas DataFrame
    - category_col: column name with category labels (y-axis)
    - left_col, right_col: numeric columns to compare
    Returns: plotly.graph_objects.Figure
    """
    # Defensive copy to protect the original dataframe of only the important information
    plot_df = df[[category_col, left_col, right_col]].copy()

    # Sort descending by left_col so highest 2019 at top
    plot_df = plot_df.sort_values(by=left_col, ascending=False).reset_index(drop=True)
    categories = plot_df[category_col].astype(str).tolist()

    fig = go.Figure()

    # Add connecting lines as separate traces so we can color by direction
    for i, row in plot_df.iterrows():
        x0 = row[left_col]; x1 = row[right_col]; y = row[category_col]
        line_color = increase_color if x1 > x0 else (decrease_color if x1 < x0 else "gray")
        fig.add_trace(go.Scatter(
            x=[x0, x1],
            y=[y, y],
            mode="markers+lines",
            marker=dict(symbol="arrow", color=line_color, size=marker_size, angleref="previous", standoff=8),
            line=dict(color=line_color, width=2),
            showlegend=False,
            hoverinfo="none"  # markers will carry hover info
        ))

    # left markers (2019)
    fig.add_trace(go.Scatter(
        x=plot_df[left_col],
        y=plot_df[category_col],
        mode="markers",
        name=left_col,
        marker=dict(color=left_color, size=marker_size, line=dict(color="black", width=0.5)),
        hovertemplate=f"{category_col}: %{{y}}<br>{left_col}: %{{x}}<extra></extra>"
    ))

    # right markers (2023)
    fig.add_trace(go.Scatter(
        x=plot_df[right_col],
        y=plot_df[category_col],
        mode="markers",
        name=right_col,
        marker=dict(color=right_color, size=marker_size, line=dict(color="black", width=0.5)),
        hovertemplate=f"{category_col}: %{{y}}<br>{right_col}: %{{x}}<extra></extra>"
    ))

    # Here we add invisible small scatter for legend clarity and layout settings
    fig.update_layout(
        xaxis_title="Total Agreement",
        yaxis=dict(autorange="reversed"),  # keep highest at the top
        template="simple_white",
        height=max(400, figsize_height_per_row * len(plot_df)),
        legend=dict(title="", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    # Add annotations showing delta and color-coded sign
    deltas = plot_df[right_col] - plot_df[left_col]
    for i, (cat, left_val, right_val, delta) in enumerate(zip(plot_df[category_col], plot_df[left_col], plot_df[right_col], deltas)):
        sign = "+" if delta > 0 else ("-" if delta < 0 else "")
        color = increase_color if delta > 0 else (decrease_color if delta < 0 else "gray")
        fig.add_annotation(
            x=max(left_val, right_val) + (0.02 * (fig.layout.xaxis.range[1] if fig.layout.xaxis.range else max(plot_df[right_col].max(), plot_df[left_col].max()))),
            y=cat,
            text=f"{sign}{delta:.2f}",
            showarrow=False,
            font=dict(color=color),
            yshift=0
        )

    # If x-axis range not set, make a comfortable padding
    x_max = max(plot_df[right_col].max(), plot_df[left_col].max())
    x_min = min(plot_df[right_col].min(), plot_df[left_col].min())
    padding = (x_max - x_min) * 0.08 if (x_max - x_min) > 0 else 1
    fig.update_xaxes(range=[max(0, x_min - padding), x_max + padding])

    return fig

In [20]:
def plot_extremes_vs_country(df, n=10):
    """
    Compare extreme city changes to their country's average trend.
    Shows how top and bottom movers deviate from national averages.
    """
    # calculate country-level mean
    country_avg = (df.groupby("Country", as_index=False)["Change in Agreement"]
                     .mean()
                     .rename(columns={"Change in Agreement": "Country Avg"}))
    
    # merge back to get deviation
    merged = df.merge(country_avg, on="Country", how="left")
    merged["Deviation"] = merged["Change in Agreement"] - merged["Country Avg"]

    # select biggest outliers (both directions)
    top = merged.nlargest(n, "Deviation")
    bottom = merged.nsmallest(n, "Deviation")
    extremes = pd.concat([top, bottom])
    extremes["City label"] = extremes["City/LAU"] + " — " + extremes["Country"]

    # plot
    plt.figure(figsize=(10, 7))
    sns.barplot(
        data=extremes.sort_values("Deviation"),
        x="Deviation", y="City label", hue="Country", dodge=False
    )
    plt.axvline(0, color="gray", ls="--", lw=1)
    plt.title("Extreme City Deviations from Country Trend (2019→2023)")
    plt.xlabel("Deviation from Country Average (%)")
    plt.ylabel("")
    plt.legend(title="Country", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

    return extremes[["Country", "City/LAU", "Change in Agreement", "Country Avg", "Deviation"]]

In [21]:
def plot_safety_residuals(merged, x_col="Tourism Δ (%)", y_col="Change in Agreement"):
    """
    Visualize how each country's perceived safety deviated from the expected trend given tourism change.
    Fits: Change in Agreement = a * Tourism Δ + b
    """
    df = merged.copy()
    df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
    df[y_col] = pd.to_numeric(df[y_col], errors="coerce")
    mask = np.isfinite(df[x_col]) & np.isfinite(df[y_col])
    fit = df.loc[mask, ["Country", x_col, y_col]].copy()

    if len(fit) < 3:
        raise ValueError(f"Not enough valid rows (have {len(fit)}).")

    # Linear fit: y = a*x + b
    a, b = np.polyfit(fit[x_col].to_numpy(), fit[y_col].to_numpy(), 1)
    print(f"slope={a:.3f}, intercept={b:.3f}, n={len(fit)}")

    fit["Predicted_Safety"] = a * fit[x_col] + b
    fit["Residual"] = fit[y_col] - fit["Predicted_Safety"]
    fit["AbsResidual"] = fit["Residual"].abs()
    fit_sorted = fit.sort_values("AbsResidual", ascending=False)

    # Plot
    plt.figure(figsize=(10, 7))
    sns.barplot(
        data=fit_sorted,
        y="Country",
        x="Residual",
        hue=np.where(fit_sorted["Residual"] > 0, "Above trend", "Below trend"),
        dodge=False,
        palette={"Above trend": "green", "Below trend": "red"},
    )
    plt.axvline(0, color="gray", lw=1, ls="--")
    plt.title("Perceived Safety vs Tourism: Countries Above/Below Expected Trend (2019→2023)")
    plt.xlabel("Residual (Actual − Predicted Safety Change %)")
    plt.ylabel("")
    plt.legend(title="Trend position")
    plt.tight_layout()
    plt.show()

    return fit_sorted.drop(columns="AbsResidual")[["Country", x_col, y_col, "Predicted_Safety", "Residual"]]


In [22]:
def plot_safety_vs_tourism(country_change, tourism_change):
    """
    Merge safety change with tourism % change and plot a scatter + trendline.
    Expects:
      country_change: ['Country', 'Change in Agreement']
      tourism_change: ['Country', 'Tourism Δ (%)']
    """
    merged = country_change.merge(
        tourism_change[['Country', 'Tourism Δ (%)']],
        on='Country', how='inner'
    ).dropna(subset=['Tourism Δ (%)', 'Change in Agreement'])

    plt.figure(figsize=(8, 6))
    sns.regplot(
        data=merged,
        x='Change in Agreement', y='Tourism Δ (%)',
        scatter_kws={'s': 70}, ci=None, color='teal'
    )

    plt.title('Perceived Safety Change vs Tourism Growth (2019→2023)')
    plt.xlabel('Change in Perceived Safety Agreement (%)')
    plt.ylabel('Change in Tourism Nights (%)')
    plt.axhline(0, color='gray', lw=1, ls='--')
    plt.axvline(0, color='gray', lw=1, ls='--')
    plt.tight_layout()
    plt.show()

    corr = merged[['Change in Agreement', 'Tourism Δ (%)']].corr().iloc[0, 1]
    print(f"Pearson correlation: {corr:.2f}  (n={len(merged)})")

    return merged

In [23]:
def plot_safety_vs_tourism_scatter(
    merged,
    x_col="Tourism Δ (%)",
    y_col="Change in Agreement",
    label_top_n=10
):
    df = merged.copy()
    df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
    df[y_col] = pd.to_numeric(df[y_col], errors="coerce")
    df = df.dropna(subset=[x_col, y_col, "Country"]).reset_index(drop=True)

    # Remove infinities just in case
    df = df[np.isfinite(df[x_col]) & np.isfinite(df[y_col])]
    print(f"✅ Cleaned dataset: {len(df)} rows")
    print(df[[x_col, y_col]].describe())

    if df.empty or df[x_col].nunique() < 3:
        raise ValueError("Not enough valid numeric data for regression.")

    # Compute slope and intercept manually (OLS)
    x = df[x_col].to_numpy(dtype=float)
    y = df[y_col].to_numpy(dtype=float)

    x_mean = np.mean(x)
    y_mean = np.mean(y)
    var_x = np.mean((x - x_mean) ** 2)
    cov_xy = np.mean((x - x_mean) * (y - y_mean))

    if np.isfinite(var_x) and var_x != 0:
        a = cov_xy / var_x
        b = y_mean - a * x_mean
    else:
        raise ValueError("Invalid data variance (check x values).")

    print(f"slope = {a:.4f}, intercept = {b:.4f}, n = {len(df)}")

    # Predictions + residuals
    df["Predicted_Safety"] = a * df[x_col] + b
    df["Residual"] = df[y_col] - df["Predicted_Safety"]
    df["Trend"] = np.where(df["Residual"] > 0, "Above trend", "Below trend")

    label_idx = df["Residual"].abs().nlargest(label_top_n).index

    # Plot
    plt.figure(figsize=(9, 7))
    sns.scatterplot(
        data=df,
        x=x_col,
        y=y_col,
        hue="Trend",
        palette={"Above trend": "green", "Below trend": "red"},
        s=80,
        edgecolor="black",
        alpha=0.9,
    )

    # Regression line
    x_vals = np.linspace(df[x_col].min(), df[x_col].max(), 100)
    plt.plot(x_vals, a * x_vals + b, color="gray", lw=2, ls="--", label="Regression line")

    # Label key countries
    for _, r in df.loc[label_idx].iterrows():
        plt.text(r[x_col] + 0.3, r[y_col], r["Country"], fontsize=8, alpha=0.85)

    plt.axhline(0, color="lightgray", lw=1)
    plt.axvline(0, color="lightgray", lw=1)
    plt.title("Perceived Safety vs Tourism Change (2019→2023)")
    plt.xlabel("Tourism Change (% in nights)")
    plt.ylabel("Change in Perceived Safety Agreement (%)")
    plt.legend(title="Relative to regression")
    plt.tight_layout()
    plt.show()

    return df[["Country", x_col, y_col, "Predicted_Safety", "Residual"]]


> End of of initialization

<hr>

#  First part: Data exploration

This Notebook serves as a script to collect to later analyze data from the Eurostat database. The findings for this notebook will be culminated in the google slide that you can find in the following link: SLIDE LINK

## Staging the dataset

- [x] Define dataset to get

- [x] Get the dataset as a pandas dataframe

- [x] Remove any unnecessary information

- [x] Test for dirty data (make the variables consistente and clear NAN)

- [x] Refactor the dataset to only hold what we are interested in

- [x] Update the dataframe to have readable names

### Extracting Eurostat data for urban perception survey

We will use data from the urban perception surveys done throughout European cities. Particularly, the data for the question: I feel safe walking alone at night? 

First step for this is to import the urban perception dataset and figure out how the data is returned by the API:

In [24]:
full_data = eurostat.get_data_df('urb_percep')
print(full_data.shape)

full_data.head(10)

(39835, 11)


,freq,indic_ur,unit,cities\TIME_PERIOD,2004,2006,2009,2012,2015,2019,2023
0,A,PS1012V,PC,AT001C,48.0,63.4,53.1,60.0,72.0,70.0,63.5
1,A,PS1012V,PC,AT002C,NaN,25.2,28.2,28.0,39.0,35.0,37.5
2,A,PS1012V,PC,BE001C,22.0,24.7,17.8,22.0,19.0,18.5,23.2
3,A,PS1012V,PC,BE002C,32.0,38.8,38.3,21.0,24.0,23.4,22.3
4,A,PS1012V,PC,BE005C,25.0,31.5,16.9,18.0,14.0,14.7,15.7
5,A,PS1012V,PC,BG001C,NaN,5.6,8.8,19.0,18.0,17.2,20.7
6,A,PS1012V,PC,BG004C,NaN,19.8,19.0,21.0,53.0,49.3,44.3
7,A,PS1012V,PC,CH002C,NaN,NaN,NaN,22.0,36.0,78.2,76.0
8,A,PS1012V,PC,CH003C,NaN,NaN,NaN,NaN,NaN,35.6,36.4
9,A,PS1012V,PC,CY001C,NaN,7.7,4.4,23.0,14.0,12.8,11.9


Given that the data uses an ID for referring to individual cities and LAUs, we shall request from the API a list of the description of the column headers to be able to collect a key for the "cities":

In [25]:
eurostat.get_dic('urb_percep')

[('freq',
  'Time frequency',
  'This code list contains the periodicity that refers to the frequency.'),
 ('indic_ur',
  'Urban audit indicator',
  'This code list defines different type of urban audit indicators.'),
 ('unit', 'Unit of measure', None),
 ('cities',
  'Geopolitical entity (declaring)',
  'This code list contains all reporting cities. A city is a local administrative unit (LAU) where at least 50 % of the population lives in one or more urban centres.')]

The description calls for a key that is called CITIES. This is a code list that is used as a standard form for multiple datasets inside the Eurostat database. With this, a dataframe will be made by downloading the TSV file that is provided from the database.

In [26]:
load_city_key("ESTAT_CITIES_3.2.tsv").head(10)

,Label - English
CODE,
BE,Belgium
BE001C,Bruxelles/Brussel (greater city)
BE001F,Bruxelles/Brussel/Leuven
BE002C,Antwerpen (greater city)
BE002F,Antwerpen
BE003C,Gent
BE003F,Gent
BE004C,Charleroi (greater city)
BE004F,Charleroi


### Isolating the points of interest

As can be seen, this dataset normalized all the results of the urban perception survey. This means that to get the results to the specific question we are looking for, we need to isolate those IDs in specific. These IDs also come from a code list like the cities key, but since we are only evaluating 5 specific entries, it is fine to just look it up.

To the right, the table with the corresponding IDs. Each ID represents the percentage of the population of a specific LAU that answers between strongly agree and strongly disagree to the question "I feel safe walking alone at night in my city".

| Answer            | ID      |
|-------------------|---------|
| Strongly agree    | PS3514V |
| Somewhat agree    | PS3515V |
| Somewhat disagree | PS3516V |
| Strongly disagree | PS3517V |

For better clarification, let us make dedicated dataframe masks to each answer, starting with strongly agree to see how it looks:

In [27]:
# Define the answer IDs and their corresponding meanings
answer_ids = {
    "Strongly agree": "PS3514V",
    "Somewhat agree": "PS3515V",
    "Somewhat disagree": "PS3516V",
    "Strongly disagree": "PS3517V"
}

# Define columns to keep
keep_cols = ["cities\TIME_PERIOD", "2004", "2006", "2009", "2012", "2015", "2019", "2023"]

# Doing a quick test to see how the dataframe looks like:
filter_by_indicator(answer_ids["Strongly agree"], keep_cols, full_data).head(10)

,cities\TIME_PERIOD,2004,2006,2009,2012,2015,2019,2023
26923,AT001C,NaN,NaN,NaN,NaN,NaN,49.2,39.7
26924,AT002C,NaN,NaN,NaN,NaN,NaN,41.5,36.0
26925,BE001C,NaN,NaN,NaN,NaN,NaN,14.7,14.4
26926,BE002C,NaN,NaN,NaN,NaN,NaN,24.3,30.7
26927,BE005C,NaN,NaN,NaN,NaN,NaN,8.2,8.4
26928,BG001C,NaN,NaN,NaN,NaN,NaN,9.4,15.3
26929,BG004C,NaN,NaN,NaN,NaN,NaN,20.1,19.7
26930,CH002C,NaN,NaN,NaN,NaN,NaN,51.4,46.7
26931,CH003C,NaN,NaN,NaN,NaN,NaN,23.8,25.3
26932,CY001C,NaN,NaN,NaN,NaN,NaN,26.1,33.3


From this test, it is possible to see that only the data for 2019 and 2023 has answers for that question, so all the rest of the columns can also be ignored.

In [28]:
# Redefine columns to keep
keep_cols = ["cities\TIME_PERIOD", "2019", "2023"]

# Making the collection of answers for each question:
answers_dfs = [
    filter_by_indicator(answer_ids["Strongly agree"], keep_cols, full_data),
    filter_by_indicator(answer_ids["Somewhat agree"], keep_cols, full_data),
    filter_by_indicator(answer_ids["Somewhat disagree"], keep_cols, full_data),
    filter_by_indicator(answer_ids["Strongly disagree"], keep_cols, full_data)
]
answers_dfs[0].head(10)

,cities\TIME_PERIOD,2019,2023
26923,AT001C,49.2,39.7
26924,AT002C,41.5,36.0
26925,BE001C,14.7,14.4
26926,BE002C,24.3,30.7
26927,BE005C,8.2,8.4
26928,BG001C,9.4,15.3
26929,BG004C,20.1,19.7
26930,CH002C,51.4,46.7
26931,CH003C,23.8,25.3
26932,CY001C,26.1,33.3


The last thing to make sure that we have what we are working for is collecting the answers into a single entry per year, so we can compare them time wise:

In [29]:
answers_2019 = combine_yearly_data(answers_dfs, list(answer_ids.keys()), "2019")
answers_2023 = combine_yearly_data(answers_dfs, list(answer_ids.keys()), "2023")

answers_2019

,cities\TIME_PERIOD,Strongly agree,Somewhat agree,Somewhat disagree,Strongly disagree
0,AT001C,49.2,32.1,14.0,3.6
1,AT002C,41.5,38.2,14.7,3.6
2,BE001C,14.7,43.2,25.9,13.7
3,BE002C,24.3,46.8,20.3,7.3
4,BE005C,8.2,34.8,33.7,21.5
...,...,...,...,...,...
100,UK004C,38.6,44.2,10.2,3.8
101,UK008C,NaN,NaN,NaN,NaN
102,UK009C,44.0,34.1,12.2,5.8
103,UK012C,39.3,43.8,8.9,4.5


In [30]:
# Cleaning up no longer necessary variables

del keep_cols
del answer_ids
del answers_dfs
del full_data

### Removing unnecessary entries and possible errors

For practicality, we will also run pandas default NaN remover in case any exist

In [31]:
answers_2019 = answers_2019.dropna(how="any")
answers_2023 = answers_2023.dropna(how="any")

# Testing if they are still the same size
if answers_2019.shape[0] == answers_2023.shape[0]:
    print("Both dataframes have the same number of rows.")
else:
    print("The dataframes have different numbers of rows.")

The dataframes have different numbers of rows.


Since it happens that both the answers for 2019 and 2023 have different places with NaNs it means the entries are no longer alligned, so we must correct that by dropping the entries that are not present in both dfs

In [32]:
answers_2019, answers_2023 = align_on_city_period(answers_2019, answers_2023)

### Transforming the dataframes

A final makeup needs to be done to make sure that the data is good to be read for humans in a table (by making the cities no longer in a code format) using the aforementioned key, and given that the key also holds information regarding which country every city is from, that will also be added to the dataframe.

In [33]:
temp_citykey = load_city_key("ESTAT_CITIES_3.2.tsv")

# Adding a Country column for identiffication
answers_2019["Country"] = answers_2019["cities\TIME_PERIOD"].str[:2]
answers_2023["Country"] = answers_2023["cities\TIME_PERIOD"].str[:2]

# Map the city codes to their English labels
answers_2019["cities\TIME_PERIOD"] = answers_2019["cities\TIME_PERIOD"].map(temp_citykey["Label - English"])
answers_2023["cities\TIME_PERIOD"] = answers_2023["cities\TIME_PERIOD"].map(temp_citykey["Label - English"])

# Map the country codes to their English labels
answers_2019["Country"] = answers_2019["Country"].map(temp_citykey["Label - English"])
answers_2023["Country"] = answers_2023["Country"].map(temp_citykey["Label - English"])

# Rename the column to "City/FUA"
answers_2019 = answers_2019.rename(columns={"cities\TIME_PERIOD": "City/LAU"})
answers_2023 = answers_2023.rename(columns={"cities\TIME_PERIOD": "City/LAU"})

# Drop the city key
del temp_citykey

The data should now be ready to be loaded into the visualizations and analysis done ahead.

## Analysis of tendencies per city

### Raw tables:

Below we have the two processed tables with the data for the urban perception survey, specifically for the percentage of total answers for each city between strongly agree, somewhat agree, somewhat disagree, and strongly disagree for the question do you feel safe walking alone at night in your city?

In [34]:
answers_2019

,City/LAU,Strongly agree,Somewhat agree,Somewhat disagree,Strongly disagree,Country
0,Wien (greater city),49.2,32.1,14.0,3.6,Austria
1,Graz,41.5,38.2,14.7,3.6,Austria
2,Bruxelles/Brussel (greater city),14.7,43.2,25.9,13.7,Belgium
3,Antwerpen (greater city),24.3,46.8,20.3,7.3,Belgium
4,Liège (greater city),8.2,34.8,33.7,21.5,Belgium
...,...,...,...,...,...,...
98,Istanbul,14.7,35.0,18.7,30.8,Türkiye
99,London,29.0,46.7,15.1,7.2,United Kingdom
100,Glasgow,38.6,44.2,10.2,3.8,United Kingdom
102,Cardiff,44.0,34.1,12.2,5.8,United Kingdom


In [35]:
answers_2023

,City/LAU,Strongly agree,Somewhat agree,Somewhat disagree,Strongly disagree,Country
0,Wien (greater city),39.7,37.1,15.5,6.7,Austria
1,Graz,36.0,37.3,19.0,5.8,Austria
2,Bruxelles/Brussel (greater city),14.4,44.2,24.8,14.7,Belgium
3,Antwerpen (greater city),30.7,40.8,18.3,8.6,Belgium
4,Liège (greater city),8.4,34.5,34.1,21.6,Belgium
...,...,...,...,...,...,...
98,Istanbul,17.5,26.2,16.5,39.6,Türkiye
99,London,19.2,39.3,26.1,12.7,United Kingdom
100,Glasgow,24.1,44.0,19.8,9.2,United Kingdom
102,Cardiff,37.6,32.2,17.5,9.5,United Kingdom


### Tendency tables

The data that was collected has the potential to shed light in some possible shifts in perceived safety. This could be indicated by a collection of total agreement. A sum of strong agree and somewhat agree data that would be indicative of people that otherwise do not inherently feel at risk while walking alone at night. Let us find the cities with most curious values: highest and lowest total agreement for 2019 and 2023 and the largest positive and largest negative change.

In [36]:
percieved_safety = rate_of_change_agreement(answers_2019, answers_2023)
percieved_safety.sort_values("Total Agreement_2019", ascending=False).head(5)

,Country,City/LAU,Total Agreement_2019,Total Agreement_2023,Change in Agreement
20,Denmark,København (greater city),93.4,86.5,-6.9
21,Denmark,Aalborg,92.2,83.3,-8.9
28,Spain,Oviedo,90.9,85.3,-5.6
64,Sweden,Stockholm (greater city),90.6,81.4,-9.2
58,Poland,Białystok,90.0,83.5,-6.5


In [37]:
percieved_safety.sort_values("Total Agreement_2019", ascending=True).head(5)

,Country,City/LAU,Total Agreement_2019,Total Agreement_2023,Change in Agreement
23,Greece,Athina (greater city),37.5,40.0,2.5
41,Italy,Roma,38.9,36.8,-2.1
5,Bulgaria,Sofia,40.1,47.7,7.6
4,Belgium,Liège (greater city),43.0,42.9,-0.1
32,France,Marseille (greater city),45.8,42.3,-3.5


In [38]:
percieved_safety.sort_values("Total Agreement_2023", ascending=False).head(5)

,Country,City/LAU,Total Agreement_2019,Total Agreement_2023,Change in Agreement
20,Denmark,København (greater city),93.4,86.5,-6.9
28,Spain,Oviedo,90.9,85.3,-5.6
66,Slovenia,Ljubljana,89.3,85.2,-4.1
7,Switzerland,Zurich (greater city),89.1,84.2,-4.9
52,Netherlands,Groningen,88.6,83.9,-4.7


In [39]:
percieved_safety.sort_values("Total Agreement_2023", ascending=True).head(5)

,Country,City/LAU,Total Agreement_2019,Total Agreement_2023,Change in Agreement
41,Italy,Roma,38.9,36.8,-2.1
23,Greece,Athina (greater city),37.5,40.0,2.5
32,France,Marseille (greater city),45.8,42.3,-3.5
42,Italy,Napoli (greater city),49.9,42.6,-7.3
4,Belgium,Liège (greater city),43.0,42.9,-0.1


In [40]:
percieved_safety.sort_values("Change in Agreement", ascending=False).head(5)

,Country,City/LAU,Total Agreement_2019,Total Agreement_2023,Change in Agreement
5,Bulgaria,Sofia,40.1,47.7,7.6
70,Türkiye,Antalya,73.4,79.4,6.0
49,Latvia,Rīga,66.1,70.0,3.9
23,Greece,Athina (greater city),37.5,40.0,2.5
16,Germany,Stuttgart,61.3,63.3,2.0


In [41]:
percieved_safety.sort_values("Change in Agreement", ascending=True).head(5)

,Country,City/LAU,Total Agreement_2019,Total Agreement_2023,Change in Agreement
73,United Kingdom,London,75.7,58.5,-17.2
76,United Kingdom,Belfast,83.1,66.6,-16.5
74,United Kingdom,Glasgow,82.8,68.1,-14.7
33,France,Bordeaux (greater city),79.4,65.1,-14.3
15,Germany,Essen,77.6,65.3,-12.3


From looking at the data, it was found that some of the more interesting places to keep an eye for are København for having the highest perceived safety both in 2019 and in 2023, Athina and Roma for having the lowest perceived safety in 2019 and 2023 respectively, and Sofia for having the largest improvement in perceived safety and London for the worst decline. 

In [42]:
curious_cities = [
    "København (greater city)",
	"Athina (greater city)",
	"Roma",
	"Sofia",
	"London"]

filter_by_city_list(answers_2019, curious_cities)

,City/LAU,Strongly agree,Somewhat agree,Somewhat disagree,Strongly disagree,Country
5,Sofia,9.4,30.7,26.8,30.4,Bulgaria
43,København (greater city),49.2,44.2,4.3,2.0,Denmark
48,Athina (greater city),8.5,29.0,26.5,34.7,Greece
67,Roma,8.2,30.7,32.1,25.2,Italy
99,London,29.0,46.7,15.1,7.2,United Kingdom


In [43]:
filter_by_city_list(answers_2023, curious_cities)

,City/LAU,Strongly agree,Somewhat agree,Somewhat disagree,Strongly disagree,Country
5,Sofia,15.3,32.4,27.3,24.1,Bulgaria
43,København (greater city),46.7,39.8,8.8,3.7,Denmark
48,Athina (greater city),10.7,29.3,24.0,35.3,Greece
67,Roma,7.2,29.6,31.3,29.8,Italy
99,London,19.2,39.3,26.1,12.7,United Kingdom


It was also noticeable that the positive changes were much more conservative throughout the cities in Europe than the negative changes:

In [44]:
percieved_safety.mean(numeric_only=True)

Total Agreement_2019    72.676623
Total Agreement_2023    67.857143
Change in Agreement     -4.819481
dtype: float64

By calculating the mean across the perceived safety through European cities, it is noticeable that there is a total effective drop. More nuance can be investigated through visualizing this data

### Visualizations

In [45]:
_dntk.DeepnoteChart(percieved_safety, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#F9732C","tooltip":true},"encoding":{"x":{"sort":"ascending","type":"nominal","field":"City/LAU","scale":{"type":"linear"}},"y":{"axis":{"title":"Percantage of people feeling safe","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Total Agreement_2019","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Total Agreement 2019","scale":{"range":["#F9732C"],"domain":["Total Agreement 2019"]}},"xOffset":{"datum":"series_0"}},"transform":[]}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#2266D3","tooltip":true},"encoding":{"x":{"sort":"ascending","type":"nominal","field":"City/LAU","scale":{"type":"linear"}},"y":{"axis":{"title":"Percantage of people feeling safe","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Total Agreement_2023","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Total Agreement 2023","scale":{"range":["#2266D3"],"domain":["Total Agreement 2023"]}},"xOffset":{"datum":"series_1"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Feeling safe walking alone","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Total Agreement 2019","Total Agreement 2023"],"seriesOrder":[0,1],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

In the interest of highlighting any specific declines in total agreement, two bar graphs will be put 'side to side' showing the raw values ordered in terms of least safe to most safe in 2019 with emphasis on the different years. This effectively should create a line of interest that will allow us to visualize the average value of total agreement. If it is true that for the whole dataset the average drops, then this line of interest should exist above the fluctuations of total agreement 2023:

In [46]:
_dntk.DeepnoteChart(percieved_safety, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#F9732C","tooltip":true},"encoding":{"x":{"sort":{"field":"Total Agreement_2019","order":"ascending"},"type":"nominal","field":"City/LAU","scale":{"type":"linear"}},"y":{"axis":{"title":"Percantage of people feeling safe","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Total Agreement_2019","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Total Agreement 2019","scale":{"range":["#F9732C"],"domain":["Total Agreement 2019"]}},"xOffset":{"datum":"series_0"}},"transform":[]}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#83AFF6","tooltip":true},"encoding":{"x":{"sort":{"field":"Total Agreement_2019","order":"ascending"},"type":"nominal","field":"City/LAU","scale":{"type":"linear"}},"y":{"axis":{"title":"Percantage of people feeling safe","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Total Agreement_2023","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Total Agreement 2023","scale":{"range":["#83AFF6"],"domain":["Total Agreement 2023"]}},"xOffset":{"datum":"series_1"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Feeling safe walking alone (2019 low to high) (Emphasis on line of interest)","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Total Agreement 2019","Total Agreement 2023"],"seriesOrder":[0,1],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

In [47]:
_dntk.DeepnoteChart(percieved_safety, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#FEBD89","tooltip":true},"encoding":{"x":{"sort":{"field":"Total Agreement_2019","order":"ascending"},"type":"nominal","field":"City/LAU","scale":{"type":"linear"}},"y":{"axis":{"title":"Percantage of people feeling safe","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Total Agreement_2019","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Total Agreement 2019","scale":{"range":["#FEBD89"],"domain":["Total Agreement 2019"]}},"xOffset":{"datum":"series_0"}},"transform":[]}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#2266D3","tooltip":true},"encoding":{"x":{"sort":{"field":"Total Agreement_2019","order":"ascending"},"type":"nominal","field":"City/LAU","scale":{"type":"linear"}},"y":{"axis":{"title":"Percantage of people feeling safe","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Total Agreement_2023","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Total Agreement 2023","scale":{"range":["#2266D3"],"domain":["Total Agreement 2023"]}},"xOffset":{"datum":"series_1"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"Feeling safe walking alone (2019 low to high) (Emphasis on fluctuations)","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Total Agreement 2019","Total Agreement 2023"],"seriesOrder":[0,1],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

Since there seems to be a clear indication that there was a drop across most cities in the data, it is important to also plot the change in in agreement more explicitly. This will be done with a dumbell plot adding more dimensionality to the visualization:

In [48]:
plot_dumbbell_agreement(percieved_safety, category_col="City/LAU").show()

## Analysis of tendencies per country

It is also interesting to represent the data points as an aggregate of the countries to see if there are any specific trends across the countries.

Let us first draw the table of perceived safety as an aggregate of country: 

In [49]:
aggregate_by_country(percieved_safety)

,Country,Total Agreement_2019,Total Agreement_2023,Change in Agreement
0,Austria,80.500000,75.050000,-5.450000
1,Belgium,57.333333,57.666667,0.333333
2,Bulgaria,55.350000,56.450000,1.100000
3,Croatia,82.000000,78.700000,-3.300000
4,Cyprus,69.700000,67.600000,-2.100000
5,Czechia,57.650000,52.350000,-5.300000
6,Denmark,92.800000,84.900000,-7.900000
7,Estonia,80.600000,77.500000,-3.100000
8,Finland,88.200000,81.500000,-6.700000
9,France,67.216667,61.400000,-5.816667


In [50]:
plot_dumbbell_agreement(aggregate_by_country(percieved_safety), category_col="Country").show()

In [51]:
#map_change(aggregate_by_country(percieved_safety))

In [52]:
#map_current(aggregate_by_country(percieved_safety), value_col="Total Agreement_2023")

## Further exploration

In [53]:
#plot_city_extremes(percieved_safety, n=8)

In [54]:
#deviation_table = plot_extremes_vs_country(percieved_safety, n=8)

In [55]:
#annual_iso = load_tourism_annual_wide("tour_occ.tsv", years=(2019, 2023))
#merged = plot_safety_vs_tourism(aggregate_by_country(percieved_safety), prepare_tourism_change(annual_iso))

In [56]:
#residuals = plot_safety_residuals(merged)

In [57]:
#scatter_data = plot_safety_vs_tourism_scatter(merged, label_top_n=25)

<hr>

# Second part: Predictions

In this second part of our data exploration, a machine learning model will be chosen for predicting total agreement (the measure of perceived agreement as defined earlier in the document) as of 2019. To best maximize the chance that our data is effective for this, we will enrich it with the addition of 2 economic and 2 population metrics . In addition, to decrease the dimensionality of our data, we will use the geographical sub-regions of Europe as defined by EuroVoc instead of using individual cities.

The economic metrics to be used are that of total economic activity by number of local units per city in 2019 (urb_cecfi) and total work place based employment per city in 2019 (urb_clma).
https://ec.europa.eu/eurostat/databrowser/view/urb_cecfi__custom_18799666/default/table?lang=en
https://ec.europa.eu/eurostat/databrowser/view/urb_clma__custom_18799514/default/table

The population metrics to be used are that of total city population as of the first of January of 2019 (urb_cpop1) and foreigners as a proportion of population per city in 2019 (urb_cpopcb).
https://ec.europa.eu/eurostat/databrowser/view/urb_cpop1__custom_18799849/default/table
https://ec.europa.eu/eurostat/databrowser/view/urb_cpopcb__custom_18799768/default/table

## Staging the ML model inputs

### Extracting keys to be used

In [58]:
eurostat.get_dic('urb_cecfi')

[('freq',
  'Time frequency',
  'This code list contains the periodicity that refers to the frequency.'),
 ('indic_ur',
  'Urban audit indicator',
  'This code list defines different type of urban audit indicators.'),
 ('cities',
  'Geopolitical entity (declaring)',
  'This code list contains all reporting cities. A city is a local administrative unit (LAU) where at least 50 % of the population lives in one or more urban centres.')]

In [59]:
eurostat.get_dic('urb_clma')

[('freq',
  'Time frequency',
  'This code list contains the periodicity that refers to the frequency.'),
 ('indic_ur',
  'Urban audit indicator',
  'This code list defines different type of urban audit indicators.'),
 ('cities',
  'Geopolitical entity (declaring)',
  'This code list contains all reporting cities. A city is a local administrative unit (LAU) where at least 50 % of the population lives in one or more urban centres.')]

In [60]:
eurostat.get_dic('urb_cpop1')

[('freq',
  'Time frequency',
  'This code list contains the periodicity that refers to the frequency.'),
 ('indic_ur',
  'Urban audit indicator',
  'This code list defines different type of urban audit indicators.'),
 ('cities',
  'Geopolitical entity (declaring)',
  'This code list contains all reporting cities. A city is a local administrative unit (LAU) where at least 50 % of the population lives in one or more urban centres.')]

In [61]:
eurostat.get_dic('urb_cpopcb')

[('freq',
  'Time frequency',
  'This code list contains the periodicity that refers to the frequency.'),
 ('indic_ur',
  'Urban audit indicator',
  'This code list defines different type of urban audit indicators.'),
 ('cities',
  'Geopolitical entity (declaring)',
  'This code list contains all reporting cities. A city is a local administrative unit (LAU) where at least 50 % of the population lives in one or more urban centres.')]

As per design, all the datasets are keyed on the same CITIES key, so we are good to continue

In [62]:
temp_citykey = load_city_key("ESTAT_CITIES_3.2.tsv")

### Extracting and transforming economic activity data

In [63]:
full_data = eurostat.get_data_df('urb_cecfi')
print(full_data.shape)

# Define columns to keep
keep_cols = ["cities\TIME_PERIOD", "2019"]
cities_ECA_2019 = filter_by_indicator("EC2039V", keep_cols, full_data)
cities_ECA_2019 = cities_ECA_2019.dropna(how="any")

# Add country corelation
cities_ECA_2019["Country"] = cities_ECA_2019["cities\TIME_PERIOD"].str[:2]

# Map the and country city codes to their English labels
cities_ECA_2019["cities\TIME_PERIOD"] = cities_ECA_2019["cities\TIME_PERIOD"].map(temp_citykey["Label - English"])
cities_ECA_2019["Country"] = cities_ECA_2019["Country"].map(temp_citykey["Label - English"])

# Rename the column to "City/FUA"
cities_ECA_2019 = cities_ECA_2019.rename(columns={"cities\TIME_PERIOD": "City/LAU"})

print(cities_ECA_2019.shape)
cities_ECA_2019.head(5)

(1339, 35)
(221, 3)


,City/LAU,2019,Country
889,France,5670332.0,France
890,Paris (greater city),1233057.0,France
891,Lyon (greater city),132571.0,France
892,Lille (greater city),76429.0,France
893,Marseille (greater city),97546.0,France


### Extracting and transforming employment data

In [64]:
full_data = eurostat.get_data_df('urb_clma')
print(full_data.shape)

# Define columns to keep
keep_cols = ["cities\TIME_PERIOD", "2019"]
cities_WPE_2019 = filter_by_indicator("EC2020V", keep_cols, full_data)
cities_WPE_2019 = cities_WPE_2019.dropna(how="any")

# Add country corelation
cities_WPE_2019["Country"] = cities_WPE_2019["cities\TIME_PERIOD"].str[:2]

# Map the city codes to their English labels
cities_WPE_2019["cities\TIME_PERIOD"] = cities_WPE_2019["cities\TIME_PERIOD"].map(temp_citykey["Label - English"])
cities_WPE_2019["Country"] = cities_WPE_2019["Country"].map(temp_citykey["Label - English"])

# Rename the column to "City/FUA"
cities_WPE_2019 = cities_WPE_2019.rename(columns={"cities\TIME_PERIOD": "City/LAU"})

print(cities_WPE_2019.shape)
cities_WPE_2019.head(5)

(27694, 39)
(605, 3)


,City/LAU,2019,Country
20376,Belgium,4799990.0,Belgium
20377,Bruxelles/Brussel (greater city),717496.0,Belgium
20378,Antwerpen (greater city),296342.0,Belgium
20379,Gent,194905.0,Belgium
20380,Charleroi (greater city),85387.0,Belgium


### Extracting and transforming population data

In [65]:
full_data = eurostat.get_data_df('urb_cpop1')
print(full_data.shape)

# Define columns to keep
keep_cols = ["cities\TIME_PERIOD", "2019"]
cities_TCP_2019 = filter_by_indicator("DE1001V", keep_cols, full_data)
cities_TCP_2019 = cities_TCP_2019.dropna(how="any")

# Add country corelation
cities_TCP_2019["Country"] = cities_TCP_2019["cities\TIME_PERIOD"].str[:2]

# Map the city codes to their English labels
cities_TCP_2019["cities\TIME_PERIOD"] = cities_TCP_2019["cities\TIME_PERIOD"].map(temp_citykey["Label - English"])
cities_TCP_2019["Country"] = cities_TCP_2019["Country"].map(temp_citykey["Label - English"])

# Rename the column to "City/FUA"
cities_TCP_2019 = cities_TCP_2019.rename(columns={"cities\TIME_PERIOD": "City/LAU"})

print(cities_TCP_2019.shape)
cities_TCP_2019.head(5)

(59827, 39)
(657, 3)


,City/LAU,2019,Country
7,Belgium,11455519.0,Belgium
8,Bruxelles/Brussel (greater city),1215289.0,Belgium
9,Antwerpen (greater city),526439.0,Belgium
10,Gent,262205.0,Belgium
11,Charleroi (greater city),202637.0,Belgium


### Extracting and transforming foreigners data

In [66]:
full_data = eurostat.get_data_df('urb_cpopcb')
print(full_data.shape)

# Define columns to keep
keep_cols = ["cities\TIME_PERIOD", "2019"]
cities_FPP_2019 = filter_by_indicator("DE2007I", keep_cols, full_data)
cities_FPP_2019 = cities_FPP_2019.dropna(how="any")

# Add country corelation
cities_FPP_2019["Country"] = cities_FPP_2019["cities\TIME_PERIOD"].str[:2]

# Map the city codes to their English labels
cities_FPP_2019["cities\TIME_PERIOD"] = cities_FPP_2019["cities\TIME_PERIOD"].map(temp_citykey["Label - English"])
cities_FPP_2019["Country"] = cities_FPP_2019["Country"].map(temp_citykey["Label - English"])

# Rename the column to "City/FUA"
cities_FPP_2019 = cities_FPP_2019.rename(columns={"cities\TIME_PERIOD": "City/LAU"})

print(cities_FPP_2019.shape)
cities_FPP_2019.head(5)

(13910, 38)
(527, 3)


,City/LAU,2019,Country
5378,Belgium,12.3,Belgium
5379,Bruxelles/Brussel (greater city),35.3,Belgium
5380,Antwerpen (greater city),21.2,Belgium
5381,Gent,14.8,Belgium
5382,Charleroi (greater city),15.4,Belgium


### Preparing total agreement table

In [67]:
cities_TPS_2019 = percieved_safety.copy().drop(["Total Agreement_2023", "Change in Agreement"],axis=1)

In [68]:
# Drop temp variables
del full_data
del keep_cols
del temp_citykey

### Loading all the data into one dataframe

In [69]:
merging_datasets = {
   # "Econ activity"   : cities_ECA_2019,
  #  "Employment"      : cities_WPE_2019,
    "Population"      : cities_TCP_2019,
    "% of foreigners" : cities_FPP_2019,
    "Total agreement" : cities_TPS_2019
}

ml_input = merge_datasets_CityCountry(merging_datasets)

ml_input = add_eurovoc_region(ml_input)

ml_input = ml_input.dropna(how="any").drop(["Country", "City/LAU"],axis=1)

ml_input

,EuroVoc Region,Population,% of foreigners,Total agreement
1,Western Europe,1215289.0,35.3,57.9
2,Western Europe,526439.0,21.2,71.1
5,Western Europe,383482.0,17.4,43.0
13,Central and Eastern Europe,1241675.0,2.1,40.1
16,Central and Eastern Europe,202434.0,2.0,70.6
32,Western Europe,686121.0,33.0,89.1
33,Western Europe,411367.0,42.1,76.7
58,Western Europe,3644826.0,18.5,66.6
59,Western Europe,1841179.0,16.4,83.9
60,Western Europe,1471508.0,26.0,87.3


## Preparations for model validation

### Preparing the data split for training our model

Given that, in the end, the amount of data that we have available to be made into a ML based predictor is very small  and unbalanced, a stratified data split will be necessary to guarantee relevant participation of the points. It will also be necessary to have repeated entries in k-folds to help maximize the amount of training data per fold.

In [83]:
test_engage = True

In [86]:
# Tokenize regions
region_dict = {
    "Northern Europe": 1,
    "Southern Europe": 2,
    "Western Europe": 3,
    "Central and Eastern Europe": 4
}

ml_input = convert_regions(ml_input, "EuroVoc Region", region_dict)

if test_engage:
    # Use the example dataset provided by Scikit learn for
    # evaluating that the functionality of the models
    from sklearn.datasets import load_diabetes
    diabetes = load_diabetes()
    X, y = diabetes.data, diabetes.target

    # Use normal k-folds
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
else:
    # Split input 'x' and target data 'y'
    y = ml_input["Total agreement"]
    X = ml_input.drop(columns=["Total agreement"])

    # Use EuroVoc Region for stratification
    stratify_labels = ml_input["EuroVoc Region"]

    # Split into folds
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)

### Defining models for comparing

<img src="image-20251108-140335.png" width="" align="" />

Based on the suggestion by the SciKit user guide, we thought of exploring these models to see if any can be successful in creating our desired predictor

In [72]:
models = {
    "Linear SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(kernel="linear"))
    ]),
    "RBF SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(kernel="rbf"))
    ]),
    "Ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge())
    ]),
    "Lasso": Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", Lasso(max_iter=10000))
    ]),
    "Random Forest": Pipeline([
        ("rf", RandomForestRegressor(random_state=42))
    ])
}

### Defining hyper-parameters for evaluating

In [73]:
param_grids = {
    "Linear SVR": {
        "svr__C": [0.001, 0.005, 0.1, 1, 10]
    },
    "RBF SVR": {
        "svr__C": [0.001, 0.005, 0.1, 1, 10],
        "svr__gamma": ["scale", 0.1, 0.01, 0.001, 0.005]
    },
    "Ridge": {
        "ridge__alpha": [0.1, 1, 10, 100]
    },
    "Lasso": {
        "lasso__alpha": [0.001, 0.005, 0.01, 0.1, 1, 10]
    },
    "Random Forest": {
        "rf__n_estimators": [10, 15, 20, 25, 30, 45, 50],
        "rf__max_depth": [None, 10, 20]
    }
}

## Cross validation across models

### Hyper-parameter search

In [87]:
best_models = {}
for name, pipeline in models.items():
    print(f"\nRunning GridSearch for {name}...")
    if test_engage:
        grid = GridSearchCV(
            pipeline,
            param_grids[name],
            cv=cv.split(X),
            scoring="r2",
            n_jobs=-1
        )        
    else:
        grid = GridSearchCV(
            pipeline,
            param_grids[name],
            cv=cv.split(X, stratify_labels),
            scoring="r2",
            n_jobs=-1
        )
    grid.fit(X, y)
    print(f"Best params for {name}: {grid.best_params_}")
    print(f"Best R2 score: {grid.best_score_:.3f}")
    best_models[name] = grid.best_estimator_


Running GridSearch for Linear SVR...
Best params for Linear SVR: {'svr__C': 1}
Best R2 score: 0.477

Running GridSearch for RBF SVR...
Best params for RBF SVR: {'svr__C': 10, 'svr__gamma': 'scale'}
Best R2 score: 0.471

Running GridSearch for Ridge...
Best params for Ridge: {'ridge__alpha': 10}
Best R2 score: 0.480

Running GridSearch for Lasso...
Best params for Lasso: {'lasso__alpha': 1}
Best R2 score: 0.480

Running GridSearch for Random Forest...
Best params for Random Forest: {'rf__max_depth': None, 'rf__n_estimators': 50}
Best R2 score: 0.431


### Voting regression (ensamble)

In [88]:
ensemble = VotingRegressor([
    ("linear_svr", best_models["Linear SVR"]),
    ("rbf_svr", best_models["RBF SVR"]),
    ("ridge", best_models["Ridge"]),
    ("lasso", best_models["Lasso"]),
    ("rf", best_models["Random Forest"])
], verbose = True)

### Results

In [89]:
if test_engage:
    ensemble_scores = cross_val_score(ensemble, X, y, cv=cv.split(X), scoring="r2")
else:
    ensemble_scores = cross_val_score(ensemble, X, y, cv=cv.split(X, stratify_labels), scoring="r2")
print("\nEnsemble results:")
print(f"Mean R2={ensemble_scores.mean():.3f}, Std={ensemble_scores.std():.3f}")

[Voting] ............... (1 of 5) Processing linear_svr, total=   0.0s
[Voting] .................. (2 of 5) Processing rbf_svr, total=   0.0s
[Voting] .................... (3 of 5) Processing ridge, total=   0.0s
[Voting] .................... (4 of 5) Processing lasso, total=   0.0s
[Voting] ....................... (5 of 5) Processing rf, total=   0.1s
[Voting] ............... (1 of 5) Processing linear_svr, total=   0.0s
[Voting] .................. (2 of 5) Processing rbf_svr, total=   0.0s
[Voting] .................... (3 of 5) Processing ridge, total=   0.0s
[Voting] .................... (4 of 5) Processing lasso, total=   0.0s
[Voting] ....................... (5 of 5) Processing rf, total=   0.1s
[Voting] ............... (1 of 5) Processing linear_svr, total=   0.0s
[Voting] .................. (2 of 5) Processing rbf_svr, total=   0.0s
[Voting] .................... (3 of 5) Processing ridge, total=   0.0s
[Voting] .................... (4 of 5) Processing lasso, total=   0.0s
[Votin

Our final ensemble model achieves a mean
R² ≈ 0.49 (Std ≈ 0.07) across 5 stratified folds.

This means that the model explains about 49% of the variance in perceived safety agreement across European urban regions.

Considering:
• the small dataset 
• the high sociopolitical complexity of the target variable (perception-based survey data)
• the feature sparsity 

an R² near 0.50 is actually strong performance.

In public opinion modeling, values of R² above 0.3 are often considered meaningful, so explaining half the variance in a small dataset indicates that the model identifies real underlying structural patterns. (Source: Divine Relevation)

In [90]:
best_models = {}

for name, pipeline in models.items():
    print(f"\nRunning GridSearch for {name}...")
    if test_engage:
        grid = GridSearchCV(
            pipeline,
            param_grids[name],
            cv=cv.split(X),
            scoring="r2",
            n_jobs=-1
        )
    else:
        grid = GridSearchCV(
            pipeline,
            param_grids[name],
            cv=cv.split(X, stratify_labels),
            scoring="r2",
            n_jobs=-1
        )
    grid.fit(X, y)
    print(f"Best params for {name}: {grid.best_params_}")
    print(f"Best R2 score: {grid.best_score_:.3f}")
    best_models[name] = grid.best_estimator_



Running GridSearch for Linear SVR...
Best params for Linear SVR: {'svr__C': 1}
Best R2 score: 0.477

Running GridSearch for RBF SVR...
Best params for RBF SVR: {'svr__C': 10, 'svr__gamma': 'scale'}
Best R2 score: 0.471

Running GridSearch for Ridge...
Best params for Ridge: {'ridge__alpha': 10}
Best R2 score: 0.480

Running GridSearch for Lasso...
Best params for Lasso: {'lasso__alpha': 1}
Best R2 score: 0.480

Running GridSearch for Random Forest...
Best params for Random Forest: {'rf__max_depth': None, 'rf__n_estimators': 50}
Best R2 score: 0.431


In [91]:
print(best_models.keys())


dict_keys(['Linear SVR', 'RBF SVR', 'Ridge', 'Lasso', 'Random Forest'])


In [92]:
# 1) Get the pipeline for Random Forest from best_models
rf_model = best_models["Random Forest"]          # Pipeline

# 2) Get the actual RandomForestRegressor from inside the pipeline
rf_estimator = rf_model.named_steps["rf"]        # RandomForestRegressor

# 3) Extract importances
importances = rf_estimator.feature_importances_

# 4) Create generic feature names that match the number of importances
n_features = len(importances)
feature_names = [f"Feature {i+1}" for i in range(n_features)]

# 5) Build a DataFrame and sort
feat_importance_df = (
    pd.DataFrame({
        "Feature": feature_names,
        "Importance": importances
    })
    .sort_values("Importance", ascending=False)
    .reset_index(drop=True)
)

feat_importance_df

,Feature,Importance
0,Feature 9,0.306499
1,Feature 3,0.276130
2,Feature 4,0.086604
3,Feature 10,0.074551
4,Feature 1,0.058708
5,Feature 6,0.057799
6,Feature 7,0.051759
7,Feature 5,0.049117
8,Feature 8,0.027041
9,Feature 2,0.011792


In [80]:
_dntk.DeepnoteChart(feat_importance_df, """{"layer":[{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#CC8BFF","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Feature","scale":{"type":"linear"}},"y":{"axis":{"format":{"type":"number","decimals":null},"formatType":"numberFormatFromNumberType"},"type":"quantitative","field":"Importance","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"type":"nominal","datum":"Importance","scale":{"range":["#CC8BFF"],"domain":["Importance"]}},"xOffset":{"datum":"series_0"}},"transform":[]}]}],"resolve":{"scale":{"color":"independent"}}}],"title":"","config":{"legend":{"disable":false}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"seriesNames":["Importance"],"seriesOrder":[0],"specSchemaVersion":2,"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

Feature 9 and 3 most likely dominate as they predominantly capture the strongest singlas in the dataset ( economic activity and regional clusters). The other features are decently important.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37c198e5-2d9d-4d28-8044-4b6f490ea356' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>